### Automatic model selection

Test notebook for automatic model selection.

In [3]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression
#from tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPS import CMAPSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
#from data_handler_MNIST import MNISTDataHandler
#from data_handler_CMAPS import CMAPSDataHandler

Using TensorFlow backend.


### Load cmaps data handler

In [4]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Test on MNIST

In [6]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
#dhandler_cmaps, input_shape = cmaps_dhandler()

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

Launching experiment 1

Generation 1
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 976)               766160    
_________________________________________________________________
dropout_43 (Dropout)         (None, 976)               0         
_________________________________________________________________
dense_191 (Dense)            (None, 10)                9770      
Total params: 775,930
Trainable params: 775,930
Non-trainable params: 0
_________________________________________________________________


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


KeyboardInterrupt: 

### Test on CMAPSS

In [9]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = (784,)
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()
print(input_shape)

#mnist datahandler
#dHandler_mnist = MNISTDataHandler()

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, epochs=5, cross_val=0.2, size_scaler=0.4,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, 
                                                    unroll=unroll, verbose_data=0, tModel_scaler=min_max_scaler)

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

(350,)
Launching experiment 1

Generation 1
Fetching to keras
Evaluating population
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 776)               272376    
_________________________________________________________________
dropout_69 (Dropout)         (None, 776)               0         
_________________________________________________________________
dense_260 (Dense)            (None, 872)               677544    
_________________________________________________________________
dropout_70 (Dropout)         (None, 872)               0         
_________________________________________________________________
dense_261 (Dense)            (None, 1)                 873       
Total params: 950,793
Trainable params: 950,793
Non-trainable params: 0
_________________________________________________________________
training with cv
Train on 14555 samples, validate on

KeyboardInterrupt: 